@author: Madhumitha Sakthi, Maansi Desai

This code processes the .fif EEG files and partitions them based on .wav file name. 

In [1]:
#import packages
import mne # For loading EEG data
import numpy as np
from praatio import tgio # For importing textgrids with word level transcriptions
import glob 
import os

from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
%matplotlib inline


In [2]:
#define subject
subjects = ['MT0001','MT0002','MT0003','MT0004','MT0005','MT0006', 'MT0008','MT0009', 'MT0010', 'MT0011', 'MT0012', 'MT0013', 'MT0014', 'MT0015', 'MT0016', 'MT0017']

data_dir = 'path/to/original/files'  #.fif files 

In [3]:
def load_raw_EEG(subject, data_dir):
    '''
    Load the preprocessed EEG (post-ICA), removing bad time points if they exist
    Input : 
        subject [str] : the subject ID (e.g. 'MT0001')
        data_dir [str] : the path to your data
    Output : 
        raw [mne Raw object] : the data structure containing the EEG for this participant
    '''
    eeg_file = '%s/%s_postICA_rejected.fif'%(data_dir, subject)
    raw = mne.io.read_raw_fif(eeg_file, preload=True)
    

    # Print which are the bad channels, but don't get rid of them yet...
    raw.pick_types(eeg=True, meg=False, exclude=[])
    bad_chans = raw.info['bads']
    print("Bad channels are: ")
    print(bad_chans)

    # Get onset and duration of the bad segments in samples
    bad_time_onsets = raw.annotations.onset * raw.info['sfreq']
    bad_time_durs = raw.annotations.duration * raw.info['sfreq']

    print(raw._data.shape)

    # Set the bad time points to zero
    for bad_idx, bad_time in enumerate(bad_time_onsets):
        raw._data[:,np.int(bad_time):np.int(bad_time+bad_time_durs[bad_idx])] = 0
    
    return raw

# Load event file and get times of TIMIT sentences or MTs
def load_event_file(subject, data_dir, fs=128, event_file_name='TIMIT'):
    '''
    Get the TIMIT or MovieTrailers event file with start and stop times relative to the EEG start time for a 
    particular subject. This event file has one row per TIMIT sentence or one row per MovieTrailer, depending
    on which stimulus is chosen.
    Inputs:
        subject [str] : subject ID (e.g. 'MT0001')
        data_dir [str] : path to your data and event files
        fs [int] : the sampling frequency for returning the samples
        event_file_name [str] : 'TIMIT' or 'MovieTrailers' -- which event file to load
    Output:
        evs : the offset and onset samples (assuming [fs] sampling frequency)
        wav_id : the name of the .wav that corresponds to ev
    '''
    if event_file_name=='TIMIT':
        event_file = '%s/%s_TIMIT_all_events.txt'%(data_dir, subject) #timit
        
    elif event_file_name=='MOVIE':
        event_file = '%s/%s_MovieTrailers_events.txt'%(data_dir, subject) #movie trailer
    
    else:
        print('Input is unrecognizable.')
   
    # Load the columns with the times    
    evs = np.loadtxt(event_file, dtype='f', usecols = (0, 1, 2))
    evs[:,:2] = evs[:,:2]*fs #128 is the downsampled frequency from EEG data
    evs = evs.astype(np.int) #this takes into account onset and offset times
    wav_id = np.loadtxt(event_file, dtype='<U', usecols = 3) #name of .wav filename
    
    return evs, wav_id


#function to epoch your data
def get_event_epoch(raw, evs, event_id, wav_id, bef_aft=[-0.5,0.5], baseline = None, reject_by_annotation=False):

    '''
    Epoch preprocessed EEG (post-ICA). Will epoch based on length of TIMIT sentence specified in event file
    Input : 
        raw [mne Raw object] : created from load_raw_EEG for given participant
        evs [numpy array] : the path to your data with all event onset information from textfiles. Also contains event_id in last column
        event_id [int]: last column in evs 
        bef_aft: hard coded to give epochs of neural data 0.5 seconds before and after
    Output : 
        raw [mne Raw object] : the data structure containing the EEG for this participant
    '''
    
    # Get duration information
    max_samp_dur = np.max(evs[(np.where(evs[:,2] == event_id)),1]-evs[(np.where(evs[:,2] == event_id)),0])
    trial_dur = max_samp_dur/raw.info['sfreq']
    
    epochs = mne.Epochs(raw, evs, event_id=[event_id], tmin=bef_aft[0], tmax=trial_dur+bef_aft[1], baseline=baseline,
                        reject_by_annotation=reject_by_annotation)
    ep = epochs.get_data()
    times = epochs.times
    wav_title = wav_id[np.where(evs[:,2] == event_id)]
        
    return ep, times, wav_title

In [ ]:
#iterate through each subject and save data individually

for subject in subjects:
    sub_dir = data_dir+'/'+ subject
    print("sub_dir:", sub_dir)
    raw = load_raw_EEG(subject, sub_dir)
    raw.interpolate_bads() # Interpolate the bad channels 
    raw.plot_psd()

    ch_names = raw.info['ch_names']

    # Get the sentence event times.
    # evs contains the start sample, stop sample, and unique number identifying the sentence
    # These unique names can be matched up to specific wav files in Stimuli/wavs and Stimuli/Textgrids
    # with wav_id
    
    #load timit data
    evs_timit, wav_id_timit = load_event_file(subject, sub_dir, fs=128, event_file_name='TIMIT')
    
    #load movie data
    evs_movie, wav_id_movie = load_event_file(subject, sub_dir, fs=128, event_file_name='MOVIE')
    
    timit_total = 0
    movie_total = 0
    timit_filename = data_dir + '/EEG_LSTM/'+ subject + 'timit/'#.npy'
    movie_filename = data_dir + '/EEG_LSTM/'+ subject + 'movie/'#.npy'
    os.mkdir(timit_filename)
    print("mkdir", timit_filename)
    os.mkdir(movie_filename)
    print("mkdir", movie_filename)
    
    
    #process timit sentences
    event_ids = evs_timit[:,2]
    print(event_ids.shape)
    event_ids = np.unique(event_ids) #extract unique sentences from event_files
    print(event_ids)


    
    
    #print(event_ids.shape[0])
    for x in range(event_ids.shape[0]):
        event_id = event_ids[x]
        
        #print("event id:", event_id)
        ep, times, wav_title = get_event_epoch(raw, evs_timit, event_id, wav_id_timit, bef_aft=[-0.5,1], baseline = (None, 0), reject_by_annotation=False)
        #print("ep shape:", ep.shape)
        timit_file = timit_filename + wav_title[0][:-4] + '.npy' 
        if ep.shape[0] >= 1:
            ep = ep[[0]]
            print("ep shape:", ep.shape, wav_title[0][:-4])
            
            timit_total = timit_total + 1
            np.save(timit_file, ep)

    print(timit_total)
    
    
    #process movie sentences
    event_ids = evs_movie[:,2]
    #print(event_ids.shape)
    event_ids = np.unique(event_ids) #extract unique sentences from event_files
    #print(event_ids)


    
    
    #print(event_ids.shape[0])
    for x in range(event_ids.shape[0]):
        event_id = event_ids[x]
        
        #print("event id:", event_id)
        ep, times, wav_title = get_event_epoch(raw, evs_movie, event_id, wav_id_movie, bef_aft=[-0.5,1], baseline = (None, 0), reject_by_annotation=False)
        #print("ep shape:", ep.shape)
        movie_file = movie_filename + wav_title[0][:-4]
        if ep.shape[0] >= 1:
            ep = ep[[0]]
            movie_total = movie_total + 1
            np.save(movie_file, ep)
    #print(movie_total)
    
            
            